In [1]:
from typing import Dict
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

In [2]:
data_dir = Path("/workspaces/pyplier/transcoding_notebooks/data/mannwhitney_conf_int")

x = pd.read_csv(data_dir / "x.csv", index_col=0).squeeze("columns")
y = pd.read_csv(data_dir / "y.csv", index_col=0).squeeze("columns")
posval = pd.read_csv(data_dir / "posval.csv", index_col=0).squeeze("columns")
negval = pd.read_csv(data_dir / "negval.csv", index_col=0).squeeze("columns")

So while the current `pyplier.AUC.mannwhitneyu_conf_int()` above does give the correct result for the
algorithm used, it does not match what is returned by R {stats} `wilcox.test()`.

So here is the R code used:

```{r}
alpha <- 1 - conf.level
mumin <- min(x) - max(y)
mumax <- max(x) - min(y)
W <- function(d) {
    dr <- c(x - d, y)
    dr <- rank(if (is.finite(digits.rank)) 
    signif(dr, digits.rank)
    else dr)
    NTIES.CI <- table(dr)
    dz <- sum(dr[seq_along(x)]) - n.x * (n.x + 
    1)/2 - n.x * n.y/2
    CORRECTION.CI <- if (correct) {
    switch(alternative, two.sided = sign(dz) * 
        0.5, greater = 0.5, less = -0.5)
    }
    else 0
    SIGMA.CI <- sqrt((n.x * n.y/12) * ((n.x + 
    n.y + 1) - sum(NTIES.CI^3 - NTIES.CI)/((n.x + 
    n.y) * (n.x + n.y - 1))))
    if (SIGMA.CI == 0) 
    warning("cannot compute confidence interval when all observations are tied", 
        call. = FALSE)
    (dz - CORRECTION.CI)/SIGMA.CI
}
wdiff <- function(d, zq) W(d) - zq
Wmumin <- W(mumin)
Wmumax <- W(mumax)
root <- function(zq) {
    f.lower <- Wmumin - zq
    if (f.lower <= 0) 
    return(mumin)
    f.upper <- Wmumax - zq
    if (f.upper >= 0) 
    return(mumax)
    uniroot(wdiff, lower = mumin, upper = mumax, 
    f.lower = f.lower, f.upper = f.upper, tol = tol.root, 
    zq = zq)$root
}
cint <- switch(alternative, two.sided = {
    l <- root(zq = qnorm(alpha/2, lower.tail = FALSE))
    u <- root(zq = qnorm(alpha/2))
    c(l, u)
}, greater = {
    l <- root(zq = qnorm(alpha, lower.tail = FALSE))
    c(l, +Inf)
}, less = {
    u <- root(zq = qnorm(alpha))
    c(-Inf, u)
})
attr(cint, "conf.level") <- conf.level
correct <- FALSE
ESTIMATE <- c(`difference in location` = uniroot(W, 
    lower = mumin, upper = mumax, tol = tol.root)$root)
```

In [3]:
# from scipy.stats import rankdata
# def mannwhitney_conf_int(x, y, conf_level = 0.95, digits_rank = np.inf)

In [4]:
conf_level = 0.95

In [5]:
alpha = 1 - conf_level
mumin = x.min() - y.max()
mumax = x.max() - y.min()

In [6]:
from scipy.stats import rankdata

In [7]:
rankdata(x)

array([ 9.5,  9.5, 42. , 37. , 34. , 20. ,  9.5, 40. , 27. , 23. ,  9.5,
       36. , 44. , 41. , 39. , 48. , 53. , 33. , 46. ,  9.5, 24. , 49. ,
       32. ,  9.5, 43. , 26. , 45. ,  9.5, 35. , 25. , 38. ,  9.5, 52. ,
        9.5, 31. , 22. , 30. ,  9.5, 54. ,  9.5, 50. , 51. , 19. ,  9.5,
        9.5, 28. , 29. , 47. ,  9.5, 21. ,  9.5,  9.5,  9.5,  9.5])

In [8]:
from rich.console import Console

console = Console()

In [9]:
console.print("Don't get murdered!")

Don't get murdered!

In [10]:
digits_rank = np.inf
correct = False
alternative = "two.sided"

In [11]:
def W(d):
    # dr <- c(x - d, y)
    dr = np.append(np.subtract(x, d), y)

    # dr <- rank(if (is.finite(digits.rank)) signif(dr, digits.rank) else dr)
    if np.isinf(digits_rank):
        dr = rankdata(dr)
    else:
        dr = rankdata(round(dr, digits_rank))

    # NTIES.CI <- table(dr)
    _, nties_ci = np.unique(dr, return_counts=True)

    # dz <- sum(dr[seq_along(x)]) - n.x * (n.x + 1)/2 - n.x * n.y/2
    dz = (
        np.sum(dr[range(len(x))])
        - (len(x) * ((len(x) + 1) / 2))
        - (len(x) * len(y) / 2)
    )

    # CORRECTION.CI <-
    #     if (correct){
    #         switch(
    #             EXPR = alternative,
    #             two.sided = sign(dz) * 0.5,
    #             greater = 0.5,
    #             less = -0.5
    #         )
    #     } else {
    #         0
    #     }
    if correct:
        if alternative == "two.sided":
            correction_ci = np.sign(dz) * 0.5
        elif alternative == "greater":
            correction_ci = 0.5
        elif alternative == "less":
            correction_ci = -0.5
    else:
        correction_ci = 0

    # SIGMA.CI <- sqrt((n.x * n.y/12) * ((n.x + n.y + 1) - sum(NTIES.CI^3 - NTIES.CI)/((n.x + n.y) * (n.x + n.y - 1))))
    sigma_ci = np.sqrt(
        (len(x) * len(y) / 12)
        * (
            (len(x) + len(y) + 1)
            - np.sum(nties_ci**3 - nties_ci)
            / ((len(x) + len(y)) * (len(x) + len(y) - 1))
        )
    )

    if sigma_ci == 0:
        console.print(
            "cannot compute confidence interval when all observations are tied"
        )

    return (dz - correction_ci) / sigma_ci

In [14]:
d = mumin
dr = np.append(np.subtract(x.values, d), y.values)
dr = rankdata(dr)

In [15]:
pd.concat([np.subtract(x, d), y])

gene
SMARCD3     0.317658
NOTCH4      0.317658
ZNF708      0.440214
SP1         0.428501
SMAD4       0.411437
              ...   
CFL2        0.064740
CFL1        0.000000
SELL        0.009852
GNGT2       0.000000
SERPINH1    0.000000
Name: LV1, Length: 5672, dtype: float64

In [16]:
np.append(np.subtract(x.values, d), y.values)

array([0.31765815, 0.31765815, 0.44021351, ..., 0.0098523 , 0.        ,
       0.        ])

In [17]:
np.append(np.subtract(list(x.values), d), list(y.values))

array([0.31765815, 0.31765815, 0.44021351, ..., 0.0098523 , 0.        ,
       0.        ])

In [18]:
rankdata(pd.concat([np.subtract(x, d), y]))

array([5627., 5627., 5660., ..., 3414., 1585., 1585.])

In [19]:
rankdata(np.append(np.subtract(x.values, d), y.values))

array([5627., 5627., 5660., ..., 3414., 1585., 1585.])

In [20]:
_, nties_ci = np.unique(dr, return_counts=True)

In [21]:
rankdata(dr)

array([5627., 5627., 5660., ..., 3414., 1585., 1585.])

In [22]:
np.unique(rankdata(dr), return_counts=True)

(array([1585., 3170., 3171., ..., 5670., 5671., 5672.]),
 array([3169,    1,    1, ...,    1,    1,    1]))

In [23]:
# dz <- sum(dr[seq_along(x)]) - n.x * (n.x + 1)/2 - n.x * n.y/2

In [24]:
dr[range(len(x))]

array([5627., 5627., 5660., 5655., 5652., 5638., 5627., 5658., 5645.,
       5641., 5627., 5654., 5662., 5659., 5657., 5666., 5671., 5651.,
       5664., 5627., 5642., 5667., 5650., 5627., 5661., 5644., 5663.,
       5627., 5653., 5643., 5656., 5627., 5670., 5627., 5649., 5640.,
       5648., 5627., 5672., 5627., 5668., 5669., 5637., 5627., 5627.,
       5646., 5647., 5665., 5627., 5639., 5627., 5627., 5627., 5627.])

In [25]:
np.sum(dr[range(len(x))])

304848.0

In [26]:
np.sum(dr[range(len(x))]) - (len(x) * ((len(x) + 1) / 2)) - (len(x) * len(y) / 2)

151677.0

In [27]:
dz = np.sum(dr[range(len(x))]) - (len(x) * ((len(x) + 1) / 2)) - (len(x) * len(y) / 2)

In [28]:
correction_ci = np.sign(dz) * 0.5

In [29]:
np.sqrt(
    (len(x) * len(y) / 12)
    * (
        (len(x) + len(y) + 1)
        - np.sum(nties_ci**3 - nties_ci) / ((len(x) + len(y)) * (len(x) + len(y) - 1))
    )
)

10881.458985130197

In [30]:
sigma_ci = np.sqrt(
    (len(x) * len(y) / 12)
    * (
        (len(x) + len(y) + 1)
        - np.sum(nties_ci**3 - nties_ci) / ((len(x) + len(y)) * (len(x) + len(y) - 1))
    )
)

In [31]:
mumin

-0.3176581514699247

In [32]:
W(mumin)

13.939031540464441

In [33]:
mumin

-0.3176581514699247

In [34]:
W(mumin)

13.939031540464441

In [35]:
type(x)

pandas.core.series.Series

In [36]:
type(np.inf)

float

In [37]:
def wdiff(d, zq):
    return W(d) - zq

In [43]:
Wmumin = W(mumin)
Wmumax = W(mumax)

In [39]:
from scipy.stats import norm

In [40]:
norm.isf(0.025)

1.9599639845400545

In [41]:
zq = norm.isf(alpha / 2)

In [44]:
f_lower = Wmumin - zq
f_lower

11.979067555924388

In [45]:
f_upper = Wmumax - zq
f_upper

-15.75558784762169

In [46]:
tol_root = 1e-4

In [47]:
from scipy import optimize

In [48]:
mumin

-0.3176581514699247

In [49]:
mumax

0.2919499490908167

In [50]:
zq

1.959963984540054

In [51]:
optimize.brentq(wdiff, mumin, mumax, zq)

0.002345855173076829

Not exactly the same as the result from R, which is 0.002313109

In [58]:
from scipy.optimize import brentq

In [59]:
def root(zq, Wmumin, Wmumax):
    f_lower = Wmumin - zq
    if f_lower <= 0:
        return mumin

    f_upper < -Wmumax - zq
    if f_upper >= 0:
        return mumax

    brentq(wdiff, mumin, mumax, zq)

In [60]:
if alternative == "two_sided":
    l = root(norm.isf(alpha / 2))
    u = root(norm.ppf(alpha / 2))
    cint = [l, u]
elif alternative == "greater":
    l = root(norm.isf(alpha))
    cint = [l, np.PINF]
elif alternative == "less":
    u = root(norm.ppf(alpha))
    cint = [np.NINF, u]
else:
    cint = [np.nan, np.nan]

In [61]:
correct = False
estimate = c(`difference in location` = uniroot(W, lower = mumin, upper = mumax, tol = tol.root)$root)

SyntaxError: invalid syntax (2151192502.py, line 2)

In [78]:
def conf_interval(
    x,
    y,
    conf_level: float = 0.95,
    tol_root: float = 1e-4,
    digits_rank: float = np.inf,
    correct: bool = False,
    alternative: str = "two_sided",
):
    alpha = 1 - conf_level
    mumin = min(x) - max(y)
    mumax = max(x) - min(y)

    def W(d: float) -> float:
        dr = np.append(np.subtract(x, d), y)

        if np.isinf(digits_rank):
            dr = rankdata(dr)
        else:
            dr = rankdata(round(dr, digits_rank))

        _, nties_ci = np.unique(dr, return_counts=True)

        dz = (
            np.sum(dr[range(len(x))])
            - (len(x) * ((len(x) + 1) / 2))
            - (len(x) * len(y) / 2)
        )

        if correct:
            if alternative == "two_sided":
                correction_ci = np.sign(dz) * 0.5
            elif alternative == "greater":
                correction_ci = 0.5
            elif alternative == "less":
                correction_ci = -0.5
        else:
            correction_ci = 0

        sigma_ci = np.sqrt(
            (len(x) * len(y) / 12)
            * (
                (len(x) + len(y) + 1)
                - np.sum(nties_ci**3 - nties_ci)
                / ((len(x) + len(y)) * (len(x) + len(y) - 1))
            )
        )

        if sigma_ci == 0:
            console.print(
                "cannot compute confidence interval when all observations are tied"
            )

        return (dz - correction_ci) / sigma_ci

    def wdiff(d, zq):
        return W(d) - zq

    Wmumin = W(mumin)
    Wmumax = W(mumax)

    def root(zq):
        f_lower = Wmumin - zq
        if f_lower <= 0:
            return mumin

        f_upper = Wmumax - zq
        if f_upper >= 0:
            return mumax

        return brentq(wdiff, mumin, mumax, zq, tol_root, maxiter=100)

    if alternative == "two_sided":
        l = root(norm.isf(alpha / 2))
        u = root(norm.ppf(alpha / 2))
        cint = (l, u)
    elif alternative == "greater":
        l = root(norm.isf(alpha))
        cint = (l, np.PINF)
    elif alternative == "less":
        u = root(norm.ppf(alpha))
        cint = (np.NINF, u)
    else:
        cint = (np.nan, np.nan)

    return cint

In [80]:
min(posval)

0.0

In [81]:
max(posval)

0.2919499490908167

In [82]:
min(negval)

0.0

In [84]:
max(negval)

0.3176581514699247

In [79]:
conf_interval(
    x=[
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
    ],
    y=[
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        2.1456996181329575e31,
    ],
)

RuntimeError: Failed to converge after 100 iterations.

In [108]:
conf_low, conf_high = conf_interval(posval, negval, alternative="greater")

In [109]:
conf_low

0.007149831510470957

In [110]:
from joblib import load

In [111]:
expected_file = load("/workspaces/pyplier/tests/data/AUC/auc_test_result.pkl")

In [112]:
expected_file

{'low': 0.0,
 'high': 0.0189180614565336,
 'auc': 0.5952222873136758,
 'pval': 0.003939670634429234}

In [113]:
from pyplier.AUC import AUC

In [119]:
labels = pd.read_csv(
    "/workspaces/pyplier/tests/data/AUC/labels.csv.gz", index_col=0
).squeeze("columns")
values = pd.read_csv(
    "/workspaces/pyplier/tests/data/AUC/values.csv.gz", index_col=0
).squeeze("columns")

In [123]:
res = AUC(labels, values)
res

{'low': 3.061262487135893e-05,
 'high': inf,
 'auc': 0.5952222873136758,
 'pval': 0.003939670634429234}

In [121]:
from json import dump

In [124]:
with open("/workspaces/pyplier/tests/data/AUC/auc_test_result.json", "w") as atr:
    dump(res, atr)

In [125]:
from pyplier.PLIERRes import PLIERResults

In [127]:
plierRes = PLIERResults.from_disk(
    "/workspaces/pyplier/tests/data/crossVal/crossval_plierres.json.gz"
)

In [130]:
priorMat = pd.read_csv(
    "/workspaces/pyplier/tests/data/crossVal/crossval_priormat.csv.gz", index_col=0
)

In [138]:
priorMat.loc[
    :,
    ["IRIS_Monocyte-Day0", "IRIS_Neutrophil-Resting", "DMAP_MONO1", "PID_IL6_7PATHWAY"],
].values.flatten()

array([0., 0., 0., ..., 0., 0., 0.])

In [47]:
np.random.Generator.uniform(low=0.0, high=1.0, size=None)

TypeError: descriptor 'uniform' of 'numpy.random._generator.Generator' object needs an argument

In [70]:
conf_interval(posval, negval, alternative="less")

less


(-inf, 0.044610369079090474)

In [67]:
assert (0.007149831510470957, np.inf) == conf_interval(
    posval, negval, alternative="greater"
)

greater


In [71]:
-np.inf

-inf

In [53]:
posval

gene
SMARCD3    0.000000
NOTCH4     0.000000
ZNF708     0.122555
SP1        0.110843
SMAD4      0.093779
SMAD3      0.014073
ZNF3       0.000000
ZKSCAN1    0.115999
ZNF354B    0.064277
ZNF583     0.038242
TBL1X      0.000000
ZNF615     0.110649
NCOA3      0.141449
NCOA1      0.118420
ZNF493     0.114002
ZNF211     0.150148
ZNF514     0.200260
ZNF26      0.092609
ZNF25      0.146562
ZNF235     0.000000
ESR2       0.042226
MAML2      0.161881
MAML3      0.091114
TEAD2      0.000000
ZNF45      0.125577
ZNF43      0.064189
ZNF597     0.145484
ZNF599     0.000000
ZFP90      0.100559
CCNC       0.044585
ZNF141     0.111004
ZNF420     0.000000
TRIM33     0.199494
MYC        0.000000
ZNF641     0.086640
MAMLD1     0.015179
ZFP37      0.073333
ZNF446     0.000000
ZNF440     0.291950
SNW1       0.000000
MED1       0.179196
MED7       0.181450
ZNF627     0.007200
ZNF485     0.000000
NR2F6      0.000000
ZNF567     0.071099
ZNF169     0.072429
NR4A2      0.148702
NR4A1      0.000000
MED27      0.01

In [63]:
negval

gene
GAS6        0.000000
MMP14       0.007516
MARCKSL1    0.000000
SPARC       0.000000
CTSD        0.000000
              ...   
CFL2        0.064740
CFL1        0.000000
SELL        0.009852
GNGT2       0.000000
SERPINH1    0.000000
Name: LV1, Length: 5618, dtype: float64